# Best Places To Go To Have Your Vacation Paid For With Hospital Visits

In [1]:
import pandas as pd
import sqlite3
import numpy as np
%matplotlib inline

In his book [*The 4-Hour Body: An Uncommon Guide to Rapid Fat-Loss, Incredible Sex, and Becoming Superhuman*](https://www.amazon.com/4-Hour-Body-Uncommon-Incredible-Superhuman-ebook/dp/B003EI2EH2?ie=UTF8&*Version*=1&*entries*=0) Tim Ferriss explained [How To Pay For A Beach Vacation With One Hospital Visit](http://fourhourworkweek.com/gear/#beach).  

There are tons of medical tourism services out there that can point you to the country for most economical treatment for your illness, be sure. What if you are just a regular vagabond who also wants to save on medical bills in case of the unexpected, or just save on your regular medical expenses? You don't want to end up in places where simple fever can kill you becaue of the lack of medical facilities, or places so expensive you walk out with huge hole in your credit card?  

In the sections below I ranked all countries and regions by cost of medical care per doctor - the more accurate definition is 'Health expenditure per capita (current US$)' divided by 'Physicians (per 1,000 people)'. So effectively you can word this as medical cost in US\$ ,000/year if you have a full time private doctor and pay for other expenditures? For the sake of simplicity and theatricality I will call it that.  

Other selection criteria are included. 1) only countries with life expectancy in the upper half of the spectrum (73 YO), and 2) those where TB detection rate is higher than 85\% (median). This cuts the list of country and regions down to 48. While it's not perfect - for example India has some awesome medical resources and cheap drugs due to loose drug patent laws (which is an awesome thing BTW), but coverage is too low for it to stay on the list - I feel like this is good enough for selecting countries with decent medical facilities in a more general sense, particularly when you are not a medical tourist.  

Just to throw in some bonus, I added the number of tourists visiting said country so you know they are probably worth visiting. 

To put everything together the columns in the table are, from left to right:  

|Country Name|  
|Physicians per 1000 people|  
|Health expenditure per capita (current US\$)|  
|Life expectancy at birth, total (years)|  
|Tuberculosis case detection rate (\%, all forms) |  
|International tourism, number of arrivals|  
|Annual medical cost per person/doctor (\$1,000)|

*All stats were from the World development indicators dataset, and the metrics were best measurements of the country in recent 5 years (since 2011).*

In [4]:
conn = sqlite3.connect('world-development-indicators/database.sqlite')
cur = conn.cursor()

In [5]:
query = """select CountryName, max(Value) spend
from Indicators a join Country b on a.CountryCode=b.CountryCode 
where IndicatorName = 'Health expenditure per capita (current US$)' and Year >= 2011 group by CountryName"""

healthcare_expense = pd.read_sql_query(con=conn, sql=query)

In [6]:
query2 = """select CountryName, max(Value) docs_dens
from Indicators a join Country b on a.CountryCode=b.CountryCode 
where IndicatorName = 'Physicians (per 1,000 people)' and Year >= 2011 group by CountryName"""

physician_pct = pd.read_sql_query(con=conn, sql=query2)

In [7]:
query3 = """select CountryName, max(Value) life_expectancy
from Indicators a join Country b on a.CountryCode=b.CountryCode 
where IndicatorName = 'Life expectancy at birth, total (years)' and Year >= 2011 group by CountryName"""

life_expectancy = pd.read_sql_query(con=conn, sql=query3)
#Median life expectancy is 73 YO

In [8]:
query4 = """select CountryName, max(Value) tourist_count
from Indicators a join Country b on a.CountryCode=b.CountryCode 
where IndicatorName = 'International tourism, number of arrivals' and Year >= 2011 group by CountryName"""

tourism = pd.read_sql_query(con=conn, sql=query4)

In [52]:
query5 = """select CountryName, max(Value) TB_detection
from Indicators a join Country b on a.CountryCode=b.CountryCode 
where IndicatorName in ('Tuberculosis case detection rate (%, all forms)')
and Year >= 2011 group by CountryName"""

tuberculosis = pd.read_sql_query(con=conn, sql=query5)
#Median detection rate is 85%

In [53]:
healthcare_tourism = physician_pct.merge(healthcare_expense,on='CountryName').merge(
    life_expectancy,on='CountryName').merge(
    tuberculosis, on='CountryName').merge(
    tourism, on='CountryName')
    
healthcare_tourism['cost_per_doc'] = healthcare_tourism['spend']/healthcare_tourism['docs_dens']
healthcare_tourism = healthcare_tourism[(healthcare_tourism['life_expectancy']>=73) &(
    healthcare_tourism['TB_detection']>=85)]

In [54]:
healthcare_tourism.sort(columns='cost_per_doc')

,CountryName,docs_dens,spend,life_expectancy,TB_detection,tourist_count,cost_per_doc
2,Armenia,2.845000,158.621557,74.561366,98,1.084000e+06,55.754502
31,Georgia,4.272000,349.903136,74.079512,89,5.392000e+06,81.906165
15,Bulgaria,3.866000,555.019796,74.465854,95,6.898000e+06,143.564355
68,Nicaragua,0.897000,154.608621,74.514756,90,1.229000e+06,172.361897
18,China,1.940000,366.863027,75.353024,91,5.772500e+07,189.104653
77,Romania,2.448000,507.678055,74.563415,94,8.019000e+06,207.384827
0,Albania,1.145000,239.577092,77.537244,89,3.156000e+06,209.237635
65,Montenegro,2.113000,502.283233,74.758366,87,1.324000e+06,237.710948
51,Latvia,3.579000,874.282297,73.980488,95,1.536000e+06,244.281167
35,High income: nonOECD,3.809098,984.757639,74.071020,85,1.708915e+08,258.527770


So to pay for a private doctor and other medical expenses, you need \$55,000 a year in Armenia while in the U.S it's only going to cost you \$3.7 million! On the region level, East European/Balkan countries seem to be the best places for vagabond who considers healthcare an important factor.  

And I must add that the low variance in annual incoming international tourists within these countries is somewhat surprising. 